# load recording

In [5]:
# import packages
%matplotlib qt
import glob as glob
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
from probeinterface import ProbeGroup, generate_tetrode
import json as json
import matplotlib.pyplot as plt
import numpy as np

# for path
#### later - make this a function that takes all folders of recorded data using animal name
animal = 'Pichu'
# folder_name = 'Bayleef_2025-01-30_11-53-57_010'
folder_name = 'Pichu_2025-08-15_11-59-23' 
# import data
data_path = fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}'

recording = se.read_openephys(data_path, stream_name = 'Record Node 103#OE_FPGA_Acquisition_Board-100.Rhythm Data')
events = se.read_openephys_event(data_path)
print(recording)
with open(glob.glob(data_path+'/**/structure.oebin', recursive = True)[0], 'r') as f:
    oebin = json.loads(f.read())

# select ephys channels only
selection = ['CH'+str(i) for i in np.arange(1,65)]
ephys_recording = recording.select_channels(selection)

# make a channel map 
channelmapoe = [40, 38, 36, 34,
                48, 46, 44, 42,
                56, 54, 52, 50,
                58, 64, 62, 60,
                63, 61, 59, 57,
                55, 53, 51, 49,
                47, 45, 43, 41,
                39, 37, 35, 33,
                25, 27, 29, 31,
                17, 19, 21, 23,
                9, 11, 13, 15,
                1, 3, 5, 7,
                4, 6, 8, 2,
                10, 12, 14, 16,
                18, 20, 22, 24,
                26, 28, 30, 32]

# subtract 1 to fit python indexing
channelmappythonic = np.array(channelmapoe) - 1

# create probegroup and set channel locations for sorting purposes
probegroup = ProbeGroup()
for i in range(16): # we have 16 tetrodes
    tetrode = generate_tetrode()
    tetrode.move([i * 300, 0]) # make lots of space between tetrodes
    probegroup.add_probe(tetrode)

probegroup.set_global_device_channel_indices(channelmappythonic) # tetrodes arranged based on channelmap
ephys_recording = ephys_recording.set_probegroup(probegroup, group_mode='by_probe')

# remove all bad channels
#### later - make this a function that gets all bad channels of a day from a json file
# bad_channel_ids = [48, 39, 33, 21, 12, 13, 15, 3, 5, 7]
bad_channel_ids = [2,4,6,8, 25]
bad_channel_ids = ['CH'+str(i) for i in bad_channel_ids]
# print(bad_channel_ids)
ephys_recording = ephys_recording.remove_channels(remove_channel_ids=bad_channel_ids)

# # preprocess (havent yet ditched disconnected channels from this!)
recording_f = spre.bandpass_filter(ephys_recording, freq_min=300, freq_max=6000)
recording_cmr = spre.common_reference(recording_f, reference='global', operator='median') 

# # plot cmr and filtered traces to see difference
w_ts = sw.plot_traces(recording_cmr, time_range=(0, 0.1), show_channel_ids=True)
w_t = sw.plot_traces(recording_f, time_range=(0, 0.1), show_channel_ids=True)

OpenEphysBinaryRecordingExtractor: 67 channels - 30.0kHz - 1 segments - 73,996,997 samples 
                                   2,466.57s (41.11 minutes) - int16 dtype - 9.23 GiB


In [11]:
ephy_path = glob.glob(fr"L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\**")
ephy_path.sort()
for folder in ephy_path:
    try:
        with open(glob.glob(folder+'/**/structure.oebin', recursive = True)[0], 'r') as f:
            oebin = json.loads(f.read())
    except:
        print(folder)


L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-12_15-55-42
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-12_19-56-54
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-13_11-57-25
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-13_15-55-33
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-13_19-57-54
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-14_11-57-38
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-14_15-59-51
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-14_16-03-14
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-16_14-08-36
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-16_14-10-23
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-16_14-24-13
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-16_19-55-25
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-17_10-53-44
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-17_11-56-39
L:\4portProb_ephys\Box1_ephys\Onix\Ephys\Onix_2025-07-17_16-03-23
L:\4portPr

In [43]:
bad_channel_ids, info = spre.detect_bad_channels(ephys_recording)


In [92]:
w_ts = sw.plot_traces(recording_cmr, time_range=(0, 1), show_channel_ids=True)
w_t = sw.plot_traces(recording_f, time_range=(0, 1), show_channel_ids=True)

# load opcon and find sync sample numbers for each event

In [44]:
# load session events
# read opcon file for that day (or combine with previous day)- Bayleef_2025-02-10_21-56-56_014
opcon_filepath = fr"L:\4portProb_ephys\Box1_ephys\{animal}\Behavior\Onix-2025-07-02-11-17-45.dat"
# opcon_filepath2 = fr"L:\4portProb_ephys\Box1_ephys\{animal}\Behavior\Onix-2025-07-01-13-17-54.dat"
names = ['event', 'value', 'sm', 'tp', 'smtime', 'eptime', 'dw', 'ww']
import pandas as pd
opcon_df = pd.read_csv(opcon_filepath, header = None, names = names)
# opcon_df = pd.concat([opcon_df, pd.read_csv(opcon_filepath2, header = None, names = names)])

# find the recording start time (in unix time) from ephys folder name 
rec_start_time = pd.to_datetime(folder_name.split('_')[1]+'_'+folder_name.split('_')[2], format='%Y-%m-%d_%H-%M-%S')

# take 5s before start of recording to see if any sync events are there
start_time = (rec_start_time - pd.Timedelta(seconds=5)).timestamp()

# subset any sync events sent from acquisition board to opcon
sync_df = opcon_df[(opcon_df['event'] == 26) & (opcon_df['eptime']>=start_time) & (opcon_df['eptime']<=start_time+3600)]

# count how many sync events are registered by opcon (should be equal to those created by the acquisition board - see next cell)
print(sync_df[sync_df.value == 0].shape[0])


1384


In [69]:
# check if all samples exist. 
# this takes time to load so only do it if reqd.
# and delete at the end of the cell once done! (or notebook crashes)
acq_samples = np.load(glob.glob(fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}\**\experiment1\recording1\continuous\OE_FPGA_Acquisition_Board-100.Rhythm Data\sample_numbers.npy')[0])

# to count how many sync events are created by ephys,
# generate the clock pulse for oe (up-down transitions)
pulse = np.ones(acq_samples.shape)*-1
for i in range(0, acq_samples.shape[0], 60000):
    pulse[i:i + 30000] = 1
print(np.where(np.diff(pulse)==2)[0].shape)
del acq_samples # done with it!

# find the last place where pulse transitions (whether it ended at up or down)
pulse_transitions = np.where((np.diff(pulse)==2)|(np.diff(pulse)==-2))[0]
last_pulse = pulse_transitions[-1]
del pulse
# this is the last place where pulse moved from 0 to 1. so this is the last sync event (26, 1)
# now we need to find the last sync event from opcon - if it exists
if sync_df.shape[0]>0:
    # create columns for sample numbers and timestamps
    sync_df.loc[:, 'sync_sample'] = last_pulse
    sync_df.loc[:, 'timestamp'] = -1

    # add pulse_transitions to opcon_df sample numbers
    # can subtract 30000 backwards from the last_pulse each time to get the sample number for each row
    sync_df.sync_sample = sync_df.sync_sample - (30000*np.arange(sync_df.shape[0], -1, -1)[1:])
    opcon_df.loc[:, 'sync_sample'] = np.nan
    opcon_df.loc[:, 'sync_sample'] = sync_df.sync_sample
    opcon_df.loc[:, 'sync_sample'] = opcon_df['sync_sample'].ffill()
    opcon_df.loc[:, 'timestamp'] = opcon_df['sync_sample']/30000
else:
    opcon_df['sync_sample'] = np.nan
    opcon_df['timestamp'] = np.nan

(1384,)


C:\Users\dlab\AppData\Local\Temp\ipykernel_15224\1048978228.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sync_df.loc[:, 'sync_sample'] = last_pulse
C:\Users\dlab\AppData\Local\Temp\ipykernel_15224\1048978228.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sync_df.loc[:, 'timestamp'] = -1
C:\Users\dlab\AppData\Local\Temp\ipykernel_15224\1048978228.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [6]:
# load all the events data from the ephys files usimg glob
events_data_path = fr'L:\4portProb_ephys\Box1_ephys\{animal}\Ephys\{folder_name}\**\experiment1\recording1\events\OE_FPGA_Acquisition_Board-100.Rhythm Data\TTL\*'
events = np.load(glob.glob(events_data_path+'full_words.npy', recursive = True)[0])
states = np.load(glob.glob(events_data_path+'states.npy', recursive = True)[0])
timestamps = np.load(glob.glob(events_data_path+'timestamps.npy', recursive = True)[0])
sample_numbers = np.load(glob.glob(events_data_path+'sample_numbers.npy', recursive = True)[0])

# assign correct names to each of the transitions observed by the acqusition board
state_names = ['np1', 'np2', 'np3', 'np4', 'lick', 'tone', 'water']
event_names = np.array([state_names[i] for i in np.abs(states)-1])

In [23]:
plt.plot(timestamps[np.where((states == 1) |(states == -1))[0]], states[np.where((states == 1) |(states == -1))[0]], 'o-')
plt.plot(timestamps[np.where((states == 2) |(states == -2))[0]], states[np.where((states == 2) |(states == -2))[0]]/2, 'o-')
plt.plot(timestamps[np.where((states == 3) |(states == -3))[0]], states[np.where((states == 3) |(states == -3))[0]]/3, 'o-')
plt.plot(timestamps[np.where((states == 4) |(states == -4))[0]], states[np.where((states == 4) |(states == -4))[0]]/4, 'o-')

# find trial-wise sample numbers

In [ ]:
# opcon_df = opcon_df[~opcon_df['sync_sample'].isna()]
# opcon_df[opcon_df.event == 21], sample_numbers[np.where((states == 1) |(states == -1))[0]][:10]
# get transition times for all np states - find all np 1 to np1, then subtract time 

# np1_transitions = np.cumsum(np.diff(sample_numbers[np.where((states == 1) |(states == -1))[0]]))
# plt.plot(sample_numbers[np.where((states == 1) |(states == -1))[0]], states[np.where((states == 1) |(states == -1))[0]], 'o-')
# plt.plot(np1_transitions, np.ones(np1_transitions.shape), 'or')

In [85]:
# take opcon data from when recording started...
opcon_df = opcon_df[opcon_df.eptime>=start_time-1]
# ...until when it ended
if sync_df.shape[0]>0:
    end_time = opcon_df[opcon_df.sync_sample == opcon_df.sync_sample.max()].iloc[0, 5] # last sync event
else: end_time = start_time+((pulse.shape[0]/30000)) # 1 second after the recording ended

def eventEquatinator(state_value, event_value, sample_numbers=sample_numbers, states=states, opcon_df = opcon_df, start_time=start_time, end_time=end_time):
    # find all the times when the nosepoke turned on/off from acq system (because that's what opcon notes, transitions)
    np_transitions = np.cumsum(np.diff(sample_numbers[np.where((states == state_value) |(states == -state_value))[0]]))

    # calculate differences between each of these transitions (in ms)
    np_ms_transitions = np.diff(np_transitions)/30

    # calculate differences between transitions from the opcon data (in ms)
    np_opcon_transitions = opcon_df[(opcon_df.event == event_value) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time)].smtime.diff()

    # remove the first row (will be NA)
    np_opcon_transitions = np_opcon_transitions[~np_opcon_transitions.isna()]

    # check if number of transitions and number of differences in transitions is equal 
    print(np_ms_transitions.shape == np_opcon_transitions.shape)

    # ?? shouldnt we do this check later?
    print(np_transitions.shape[0] == opcon_df[(opcon_df.event == event_value) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time)].shape[0])
    return np_transitions

# based on hardware wiring
np1_transitions = eventEquatinator(1, 21)
np2_transitions = eventEquatinator(2, 20)
np3_transitions = eventEquatinator(3, 19)
np4_transitions = eventEquatinator(4, 18)
transitions = [np1_transitions, np2_transitions, np3_transitions, np4_transitions]

opcon_df['sample_number'] = np.nan
# try to assign np transitions to opcon df in a selection of time and events 
for ind, event_val in enumerate([21, 20, 19, 18]):
    try:
        opcon_df.loc[(opcon_df.event == event_val) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time), 'sample_number'] = transitions[ind]
    except:
        # if the dumb shit doesn't work
        # find where the mismatch is, i.e. find out which of the nosepokes events didnt match
        opcon_df.loc[(opcon_df.event == event_val) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time), 'sample_number'] = np.nan
        print(f'mismatched port {ind+1}', transitions[ind].shape,
              opcon_df[(opcon_df.event == event_val) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time)].shape)
        
        # try and remove duplicate event events to see if that helps 
        mask = opcon_df.duplicated(subset = ['event', 'smtime', 'eptime'], keep = 'first')
        # remove as many duplicates?
        opcon_df.loc[(~mask) & (opcon_df.event == event_val) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time), 'sample_number'] = transitions[ind]

        continue # right now it doesnt do anything, just assigns as -1
        # then find which event didnt match based on the difference between transitions in a tolderance of lets say, 10 ms
        # something like this?
        # np.isclose((np.diff(sample_numbers[np.where((states ==4) |(states == -4))[0]])/30),
        #    opcon_df[(opcon_df.event == event_val) & (opcon_df.eptime>=start_time) &(opcon_df.eptime <= end_time)].smtime.diff().dropna(), atol = 5)
        # assign that event as NA and match all others


# propagate
opcon_df['sample_number'] = opcon_df['sample_number'].ffill(limit = 5)
# propagate to 81s only: only next 5 nans are filled

False
False
True
True
True
True
True
True
mismatched port 1 (610,) (617, 11)


In [86]:
def ephys_trializer_v3(df, list_sessionMarker,
                 trialStartMarker, trialHitMarker,
                 trialMissMarker, sessionStartMarker,
                 trialProbMarker, rewardProbMarker, animal, arms = 4):
    
    ''' Function to convert a df of joined .dat files with column names in order :
    'event', 'value', 'sm', 'tp', 'smtime', 'eptime', 'dw', 'ww'
    
    INPUTS: df, list of session state machine numbers, trial start marker, trial rewarded marker,
    trial miss/unrewarded marker, session start marker, trial probability marker, session reward prob marker, 
    animal name, num arms in task
    
    OUTPUT: single trial-wise dataframe of one animal
    2023-10-26
    '''
    # initialize var
    sessnum = 0
    npcounter = 0
    sess_list = []
    rewprobl = []

    event_lookup = {
        trialHitMarker: 1,
        trialMissMarker: 0,
    }
    list_events = [trialStartMarker, trialHitMarker, trialMissMarker, trialProbMarker, sessionStartMarker,
                   trialProbMarker, rewardProbMarker]

    # filter events as np array to increase speed and avoid repetitive indexing
    filtered_events = df[(df['sm'].isin(list_sessionMarker)) & (df['event'].isin(list_events))].to_numpy()
    indices = np.where((filtered_events[:, 0] == trialStartMarker) | (filtered_events[:, 0]==sessionStartMarker) | (filtered_events[:, 0]==rewardProbMarker))[0]

    for ind in indices:
        # extract information from the current trialStartMarker
        trialstart = filtered_events[ind, 4]
        port = filtered_events[ind, 1]
        datetime = filtered_events[ind, 5]
        task = filtered_events[ind, 2]
        event = filtered_events[ind, 0]
        sync_sample = filtered_events[ind, 8]
        sample_number = filtered_events[ind, 10]

        # select events occurring after trialStartMarker
        nextEvents = filtered_events[ind:]

        try:
            # find the index of trialMissMarker or trialHitMarker in nextEvents
            status_indices = np.where((nextEvents[:, 0] == trialMissMarker) | (nextEvents[:, 0] == trialHitMarker))[0]
            statusIndex = status_indices[0] if status_indices.size > 0 else None

            # find the index of trialProbMarker in nextEvents
            rewprob_index = np.where(nextEvents[:, 0] == trialProbMarker)[0][0] 
            #contLine: if np.any(nextEvents[:, 0] == trialProbMarker) else None

            if statusIndex is not None:
                # find reward, trialend, and rewprob based on identified events
                reward = event_lookup[nextEvents[statusIndex, 0]]
                trialend = nextEvents[statusIndex, 4]
                rewprob = nextEvents[rewprob_index, 1] if rewprob_index is not None else np.nan
            else:
                # if trialMissMarker or trialHitMarker not found, nan
                reward = np.nan
                trialend = np.nan
                rewprob = np.nan
        except IndexError:
            # set values to nan in case of issues
            reward = np.nan
            trialend = np.nan
            rewprob = np.nan

        rewprobl_str = str(np.array(rewprobl, dtype = int))
        # append to sess_list
        sess_list.append([npcounter, trialstart, port, reward, trialend, sessnum, rewprob, datetime, task, event, rewprobl_str, sync_sample, sample_number])
        npcounter += 1

        if filtered_events[ind, 0] == sessionStartMarker:
            # update session number when sessionStartMarker found
            rewprobl = []
            sessnum += 1

        if filtered_events[ind, 0] == rewardProbMarker:
            rewprobl.append(filtered_events[ind, 4])

    # create pandas dataframe from the list of all session data
    sessdf =  pd.DataFrame(sess_list, columns = ['trial', 'trialstart',
                                   'port', 'reward', 'trialend',
                                   'session', 'rewprob', 'eptime', 'task', 'event', 'rewprobfull', 'sync_sample', 'sample_number'])

    # remove all lines corresponding to start of a session or reward prob marker
    sessdf = sessdf[(sessdf.event!=sessionStartMarker) & (sessdf.event!=rewardProbMarker)].reset_index(drop = True).drop(columns = 'event')

    # remove all lines that have (trialend-trialstart)> 5500 or negative (sum of tone + wait time)
    # is this check required?

    # reset trial numbers
    sessdf['trial'] = np.arange(len(sessdf))

    # add animal information
    sessdf['animal'] = animal
    
    # add datetime
    sessdf['datetime'] = pd.to_datetime(sessdf.eptime, unit='s')

    return sessdf

sessdf = ephys_trializer_v3(opcon_df[opcon_df.eptime<=end_time], [13], 81, 51, 86, 61, 88, 83, animal) # just to see exactly how many trials there were
sessdf # tada! events matched with sample numbers

,trial,trialstart,port,reward,trialend,session,rewprob,eptime,task,rewprobfull,sync_sample,sample_number,animal,datetime
0,0,70141.0,4.0,0,72012.0,1,15.0,1.751486e+09,13.0,[46 80 46 15],9179999.0,9209644.0,Onix,2025-07-02 20:01:21
1,1,73366.0,3.0,0,75094.0,1,46.0,1.751486e+09,13.0,[46 80 46 15],9269999.0,9306415.0,Onix,2025-07-02 20:01:24
2,2,78319.0,3.0,1,79980.0,1,46.0,1.751486e+09,13.0,[46 80 46 15],9419999.0,9454987.0,Onix,2025-07-02 20:01:29
3,3,84277.0,3.0,0,85254.0,1,46.0,1.751486e+09,13.0,[46 80 46 15],9599999.0,9633753.0,Onix,2025-07-02 20:01:35
4,4,86820.0,3.0,1,88421.0,1,46.0,1.751486e+09,13.0,[46 80 46 15],9659999.0,9710043.0,Onix,2025-07-02 20:01:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,521,920508.0,1.0,1,921536.0,3,80.0,1.751489e+09,13.0,[80 46 10 15],78209999.0,78244370.0,Onix,2025-07-02 20:39:42
522,522,925361.0,1.0,1,926654.0,3,80.0,1.751489e+09,13.0,[80 46 10 15],78359999.0,78389958.0,Onix,2025-07-02 20:39:47
523,523,4096.0,1.0,0,5077.0,4,15.0,1.751489e+09,13.0,[15 80 10 46],78509999.0,78553103.0,Onix,2025-07-02 20:39:52
524,524,6548.0,1.0,0,7698.0,4,15.0,1.751489e+09,13.0,[15 80 10 46],78599999.0,78626653.0,Onix,2025-07-02 20:39:55


In [89]:
sessdf[sessdf.sample_number.isna()]

,trial,trialstart,port,reward,trialend,session,rewprob,eptime,task,rewprobfull,sync_sample,sample_number,animal,datetime
37,37,196616.0,1.0,0,197748.0,1,46.0,1.751487e+09,13.0,[46 80 46 15],12959999.0,NaN,Onix,2025-07-02 20:03:27


In [87]:
opcon_df.loc[opcon_df.eptime>=1751486607.0]

,event,value,sm,tp,smtime,eptime,dw,ww,sync_sample,timestamp,sample_number
27998,25,0,13,0,195780,1751486607,702,702,12929999.0,430.999967,NaN
27999,26,0,13,0,195821,1751486607,702,702,12959999.0,431.999967,NaN
28000,21,1,13,0,196615,1751486607,702,702,12959999.0,431.999967,13003965.0
28001,22,1,13,0,196615,1751486607,702,702,12959999.0,431.999967,13003965.0
28002,21,0,13,0,196615,1751486607,702,702,12959999.0,431.999967,13003965.0
...,...,...,...,...,...,...,...,...,...,...,...
48204,22,0,17,0,861019,1751541261,0,2000,83039999.0,2767.999967,NaN
48205,18,1,17,0,861118,1751541261,0,2000,83039999.0,2767.999967,NaN
48206,22,1,17,0,861118,1751541261,0,2000,83039999.0,2767.999967,NaN
48207,18,0,17,0,861197,1751541261,0,2000,83039999.0,2767.999967,NaN


In [37]:
# align accelero data to nosepoke4
from scipy.signal import resample
acc_channels = ['A1_AUX1']
port = 3
acc_data = recording.select_channels(acc_channels)
resampling_rate = 300
sampling_rate = 30000
pre_events = 0.5*sampling_rate
post_events = 0.5*sampling_rate
plt.figure(figsize=(10, 5))
for port in [1,2,3,4]:
    np1_times = sessdf[(sessdf.eptime<end_time)&(sessdf.port == port)].sample_number.values

    # Extract and align data around events
    aligned_data = []
    for event in np1_times:
        start = event - pre_events
        end = event + post_events
        if start >= 0 and end < acc_data.get_num_frames():
            seg = acc_data.get_traces(start_frame=start, end_frame=end, channel_ids=acc_channels)
            aligned_data.append(seg)

    # Convert to NumPy array for easier manipulation
    aligned_data = np.array(aligned_data)
    print(aligned_data.shape)


    # plt.figure(figsize=(10, 5))
    # for segment in aligned_data:
    #     plt.plot(segment, alpha=0.3, c = 'grey')
    plt.plot(np.mean(aligned_data, axis=0), label = port)
    plt.axvline(pre_events, color='k', linestyle='--')
    plt.xlabel('Time (s)')
    plt.xticks(ticks = [0, pre_events, pre_events+ post_events], labels = [-pre_events/sampling_rate, 0, post_events/sampling_rate])
    plt.title(f'Aligned accelerometer data around nosepoke {port}')
    plt.show()
plt.legend()
plt.tight_layout()

(0,)


c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(169, 30000, 1)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dlab\AppData\Local\Temp\ipykernel_15224\4101791914.py", line None, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\IPython\core\interactiveshell.py", line 2173, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\IPython\core\ultratb.py", line 1182, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dlab\miniforge3\envs\si_env_rolling_new\Lib\site-packages\IPython\core\ultra

# align units to behavioral events

In [74]:
sample_idx = np.load(r'L:\Spikesorted\Onix\Analysis_Onix_2025-07-02_19-56-14\phy_output\group_2_phy\spike_times.npy')
unit_idx = np.load(r'L:\Spikesorted\Onix\Analysis_Onix_2025-07-02_19-56-14\phy_output\group_2_phy\spike_clusters.npy')

In [353]:
from spikeinterface import load_sorting_analyzer
# 0, 2, 5, 6, 8 have all 4. best unit on 7
tet = 6
analyzer = load_sorting_analyzer(
    folder=fr"L:\ephysSorted\Bayleef_2025-01-29_11-58-18_007\analyzer_output\{tet}.zarr"
)
sw.plot_autocorrelograms(analyzer, unit_ids=analyzer.sorting.get_unit_ids())
sw.plot_isi_distribution(analyzer.sorting, unit_ids=analyzer.sorting.get_unit_ids())
sw.plot_unit_waveforms(
    analyzer,
    unit_ids=analyzer.sorting.get_unit_ids(),
    plot_legend=True, plot_templates=False, plot_channels=True)

In [348]:
ext_wf = analyzer.get_extension("waveforms")
for unit_id in analyzer.unit_ids:
    wfs = ext_wf.get_waveforms_one_unit(unit_id)
    print(unit_id, ":", wfs.shape)

ext_templates = analyzer.get_extension("templates")

av_templates = ext_templates.get_data(operator="average")
print(av_templates.shape)

median_templates = ext_templates.get_data(operator="median")
print(median_templates.shape)

for unit_index, unit_id in enumerate(analyzer.unit_ids):
    fig, ax = plt.subplots()
    template = median_templates[unit_index]
    # template = np.append(template, np.nan)
    ax.plot(template, label = f'{unit_id}')
    ax.set_title(f"{unit_id}")

# ax.legend()

# plt.figure()
# plt.plot(median_templates[2, :, 0])

# plt.plot(ext_wf.get_waveforms_one_unit(1))

1 : (500, 90, 3)
2 : (129, 90, 3)
3 : (500, 90, 3)
4 : (500, 90, 3)
(4, 90, 3)
(4, 90, 3)


In [88]:
# w_rs = sw.plot_rasters(sorting, time_range=(0, 60))

# sorting = analyzer.sorting
# import seaborn as sns
# sample_idx = sorting.to_spike_vector()['sample_index']
# unit_idx = sorting.to_spike_vector()['unit_index']
sampling_rate = 30000
pre_events = 2*sampling_rate
post_events = 2*sampling_rate
unitA = 2
spikeT_A = sample_idx[unit_idx == unitA]
eventst = []
np_selected_times = sessdf.sample_number.values[:400]
for event in np_selected_times:
    start = event - pre_events
    end = event + post_events
    if start >= 0 and end < ephys_recording.get_num_frames():
        eventst.append(spikeT_A[np.where((spikeT_A > start) & (spikeT_A < end))[0]]-event)    

color = {1: 'b', 2: 'r', 3: 'g', 4: 'orange'}
fig = plt.figure()
ax = plt.subplot(3, 1, (1, 2))
ax.eventplot(eventst, linelengths = 2.5, colors = [color[i] for i in sessdf.port.values[:400]])
ax.axvline(0, color = 'k', linestyle = '-', alpha = 0.2)
ax.set_ylabel('Trials')
ax.set_xticks(ticks = np.linspace(-60000, 60000, num = 5), labels = np.linspace(-60000, 60000, num = 5)/sampling_rate)
import matplotlib.patches as mpatches

red_patch = mpatches.Patch(color='red', label='Port 2')
blue_patch = mpatches.Patch(color='blue', label='Port 1')
green_patch = mpatches.Patch(color='green', label='Port 3')
orange_patch = mpatches.Patch(color='orange', label='Port 4')

plt.legend(handles=[blue_patch, red_patch, green_patch, orange_patch], loc='upper right')

# ax.legend(color)
# sns.despine(offset = 0, trim = True)
plt.tight_layout()
ax = plt.gca()
ax.yaxis.set_inverted(True)


# make psth along with events
ax = plt.subplot(3, 1, 3, sharex = ax)
time_bin_size = 40 # ms
bins = (pre_events + post_events)/(time_bin_size*30)
eventst_flat = [item for sublist in eventst for item in sublist]
ax.hist(eventst_flat, bins = int(bins), alpha = 0.5, color = 'k')
ax.set_ylabel(f'Spikes per {time_bin_size} ms')
ax.set_xlabel('Time (s)')
# plt.suptitle(f'Unit {np.unique(sorting.get_unit_ids())[unitA]}')
# sns.despine()
plt.tight_layout()



ValueError: colors and positions are unequal sized sequences

In [ ]:
# sort by port, then by length of trial (trialend - trialstart)
sessdf['trial_len'] = sessdf['trialend'] - sessdf['trialstart']
order = sessdf[~sessdf.sample_number.isna()].sort_values(by = ['port', 'trial_len']).reset_index(drop = True).trial

# sorting = analyzer.sorting
# sample_idx = sorting.to_spike_vector()['sample_index']
# unit_idx = sorting.to_spike_vector()['unit_index']
sampling_rate = 30000
pre_events = 2*sampling_rate
post_events = 2*sampling_rate
unitA = 1

reordered_data = sessdf.iloc[order]

spikeT_A = sample_idx[unit_idx == unitA]
eventst = []
np_selected_times = reordered_data.sample_number
for event in np_selected_times:
    start = event - pre_events
    end = event + post_events
    if start >= 0 and end < ephys_recording.get_num_frames():
        eventst.append((spikeT_A[np.where((spikeT_A > start) & (spikeT_A < end))[0]]-event)/sampling_rate)   

color = {1: 'b', 2: 'r', 3: 'g', 4: 'orange'}
fig = plt.figure()
ax = plt.subplot(3, 1, (1, 2))
ax.eventplot(eventst, linelengths = 2.5, colors = [color[i] for i in reordered_data.port])
ax.scatter(reordered_data.trial_len/1000, np.arange(reordered_data.shape[0]), color = 'k', s=1, marker = '|', alpha = 0.5)
# for i in sessdf[~sessdf.sample_number.isna()].groupby('session').head(1).trial:
#     ax.axhline(i, color = 'k', linestyle = '-', alpha = 0.2)
ax.axvline(0, color = 'k', linestyle = '-', alpha = 0.2)
ax.set_ylabel('Trials')
ax.set_ylim(0)
ax.set_xlim(-2, 2)
# ax.set_xticks(ticks = np.linspace(-60000, 60000, num = 5), labels = np.linspace(-60000, 60000, num = 5)/sampling_rate)
import matplotlib.patches as mpatches

red_patch = mpatches.Patch(color='red', label='Port 2')
blue_patch = mpatches.Patch(color='blue', label='Port 1')
green_patch = mpatches.Patch(color='green', label='Port 3')
orange_patch = mpatches.Patch(color='orange', label='Port 4')

plt.legend(handles=[blue_patch, red_patch, green_patch, orange_patch], loc='lower left')

# ax.legend(color)
# sns.despine(offset = 0, trim = True)
plt.tight_layout()
ax = plt.gca()
ax.yaxis.set_inverted(True)
# make psth along with events
ax = plt.subplot(3, 1, 3, sharex = ax)
time_bin_size = 40 # ms
bins = (pre_events + post_events)/(time_bin_size*30)
eventst_flat = [item for sublist in eventst for item in sublist]
# ax.hist(np.array(eventst_flat), bins = int(bins), alpha = 0.5, color = 'k', weights = np.ones(len(eventst_flat))/0.04)
ax.set_ylabel(f'Spike rate (Hz)')
ax.set_xlabel('Time (s) from nose-port entry')

for n in np.arange(1,5):
    eventst_temp = [eventst[i] for i in reordered_data[reordered_data.port == n].trial.values]
    eventst_flat = [item for sublist in eventst_temp for item in sublist]
    # ax.hist(eventst_flat, bins = int(bins), alpha = 0.2, color = color[n], label = f'Port {n}')
    c, _ = np.histogram(eventst_flat, bins = int(bins))
    arr = (c/reordered_data[reordered_data.port == n].shape[0])/(time_bin_size/1000)  
    ax.plot(np.linspace(-2, 2, arr.shape[0]), arr, color = color[n], label = f'Port {n}')

# plt.suptitle(f'Unit {np.unique(sorting.get_unit_ids())[unitA]}')
# sns.despine()
plt.tight_layout()


IndexError: list index out of range

In [103]:
eventst

[array([], dtype=float64),
 array([1.11226667]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([-1.95106667, -1.69526667, -0.87296667, -0.8529    ]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([-1.88803333,  1.87853333]),
 array([-1.94133333, -1.43383333,  1.41433333,  1.85476667]),
 array([1.4443, 1.645 ]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([], dtype=float64),
 array([1.75863333, 1.83406667]),
 array([], dtype=float64),
 array([-1.76083333]),
 array([1.94266667]),
 array([], dtype=float64),
 array([-1.7522    , -1.55373333,  1.20376667]),
 array([], dtype=float64),
 array([-0.91073333]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([-1.24193333]),
 array([], dtype=float64),
 array([1.65173333]),
 array([], dtype=float64),
 array([-1.0769]),
 array([], dtype=float64),
 array([], dtype=float64),
 array([-0

In [102]:
reordered_data[reordered_data.port == n].trial.values.shape, eventst[3].shape

((288,), (0,))

In [227]:
# sort by session, port chosen, then by length of trial (trialend - trialstart)
sessdf['trial_len'] = sessdf['trialend'] - sessdf['trialstart']
order = sessdf[~sessdf.sample_number.isna()].sort_values(by = ['port', 'trial_len']).reset_index(drop = True).trial

sorting = analyzer.sorting
sample_idx = sorting.to_spike_vector()['sample_index']
unit_idx = sorting.to_spike_vector()['unit_index']
sampling_rate = 30000
pre_events = 2*sampling_rate
post_events = 2*sampling_rate
unitA = 1

reordered_data = sessdf.iloc[order]

spikeT_A = sample_idx[unit_idx == unitA]
eventst = []
np_selected_times = reordered_data.sample_number
for event in np_selected_times:
    start = event - pre_events
    end = event + post_events
    if start >= 0 and end < ephys_recording.get_num_frames():
        eventst.append(spikeT_A[np.where((spikeT_A > start) & (spikeT_A < end))[0]]-event)    

color = {1: 'b', 2: 'r', 3: 'g', 4: 'orange'}
fig = plt.figure()
ax = plt.subplot(3, 1, (1, 2))
# make heatmap from eventplot


# ax.eventplot(eventst, linelengths = 2.5, colors = [color[i] for i in reordered_data.port])
# for i in sessdf[~sessdf.sample_number.isna()].groupby('session').head(1).trial:
#     ax.axhline(i, color = 'k', linestyle = '-', alpha = 0.2)
# ax.axvline(0, color = 'k', linestyle = '-', alpha = 0.2)
# ax.set_ylabel('Trials')
# ax.set_ylim(0)
# ax.set_xticks(ticks = np.linspace(-60000, 60000, num = 5), labels = np.linspace(-60000, 60000, num = 5)/sampling_rate)
# import matplotlib.patches as mpatches

# red_patch = mpatches.Patch(color='red', label='Port 2')
# blue_patch = mpatches.Patch(color='blue', label='Port 1')
# green_patch = mpatches.Patch(color='green', label='Port 3')
# orange_patch = mpatches.Patch(color='orange', label='Port 4')

# plt.legend(handles=[blue_patch, red_patch, green_patch, orange_patch], loc='upper right')

# # ax.legend(color)
# # sns.despine(offset = 0, trim = True)
# plt.tight_layout()
# ax = plt.gca()
# ax.yaxis.set_inverted(True)
# # make psth along with events
# ax = plt.subplot(3, 1, 3, sharex = ax)
# time_bin_size = 40 # ms
# bins = (pre_events + post_events)/(time_bin_size*30)
# eventst_flat = [item for sublist in eventst for item in sublist]
# # ax.hist(eventst_flat, bins = int(bins), alpha = 0.5, color = 'k')
# ax.set_ylabel(f'Spikes per {time_bin_size} ms')
# ax.set_xlabel('Time (s)')

# for n in np.arange(1,5):
#     eventst_temp = [eventst[i] for i in reordered_data[reordered_data.port == n].trial.values]
#     eventst_flat = [item for sublist in eventst_temp for item in sublist]
#     ax.hist(eventst_flat, bins = int(bins), alpha = 0.2, color = color[n], label = f'Port {n}')

plt.suptitle(f'Unit {np.unique(sorting.get_unit_ids())[unitA]}')
# sns.despine()
plt.tight_layout()


# pending - read filenames from json for animal

In [434]:
# read a json file about the animal's behavior
import json
json_filepath = fr'L:\4portProb_ephys\Box1_ephys\{animal}\bephy.json'
with open(json_filepath) as f:
    bephy = json.load(f)

In [451]:
for sess in range(len(bephy['recording_details']['sessions_list'])):
    print(bephy['recording_details']['sessions_list'][sess]['session_data']['ephys']['folder_name'])
    print(bephy['recording_details']['sessions_list'][sess]['session_data']['associated_opcon'])


Bayleef_2025-01-29_11-58-18_007
Bayleef-2025-01-29-10-52-16.dat
Bayleef_2025-01-29_19-08-05_009
Bayleef-2025-01-29-19-07-01.dat
Bayleef_2025-01-29_19-08-05_009
Bayleef-2025-01-30-11-32-00.dat
Bayleef_2025-01-30_19-02-46_012
Bayleef-2025-01-30-11-32-00.dat
Bayleef_2025-01-31_11-54-31_013
Bayleef-2025-01-31-11-32-00.dat
Bayleef_2025-01-31_11-54-31_013
Bayleef-2025-01-31-11-32-00.dat
Bayleef_2025-01-31_11-54-31_013
Bayleef-2025-02-03-11-32-00.dat
Bayleef_2025-02-03_18-51-15_003
Bayleef-2025-02-03-11-32-00.dat
Bayleef_2025-02-04_09-52-25_004
Bayleef-2025-02-03-20-30-37.dat
Bayleef_2025-02-04_13-54-50_005
Bayleef-2025-02-03-20-30-37.dat
Bayleef_2025-02-04_18-06-25_008
Bayleef-2025-02-03-20-30-37.dat
Bayleef_2025-02-05_09-58-01_009
Bayleef-2025-02-04-20-30-37.dat
Bayleef_2025-02-05_13-57-52_010
Bayleef-2025-02-04-20-30-37.dat
Bayleef_2025-02-05_17-56-49_011
Bayleef-2025-02-04-20-30-37.dat
Bayleef_2025-02-05_21-54-27_012
Bayleef-2025-02-05-21-49-54.dat
Bayleef_2025-02-06_13-55-52_013
Bayleef-